In [6]:
!pip install qiskit_ibm_runtime

In [7]:
import numpy as np
import time
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# IBM Quantum imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


# ===============================================
# CONFIGURACIÓN IBM QUANTUM
# ===============================================
def setup_ibm_quantum(token=None, instance=None):
    """
    Configura el servicio de IBM Quantum.
    
    Args:
        token: Tu IBM Quantum API token (obtenerlo de https://quantum.ibm.com/)
        instance: Tu hub/group/project (ej: 'ibm-q/open/main')
    """
    if token:
        # Primera vez: guardar credenciales
        QiskitRuntimeService.save_account(
            channel="ibm_cloud",
            token=token,
            overwrite=True,
            instance=instance
        )
        print("✅ Credenciales guardadas exitosamente")
    
    # Cargar servicio
    service = QiskitRuntimeService(channel="ibm_cloud")
    
    # Mostrar backends disponibles
    print("\n📡 Backends disponibles:")
    backends = service.backends()
    for i, backend in enumerate(backends[:5], 1):
        status = backend.status()
        print(f"   {i}. {backend.name}")
        print(f"      - Qubits: {backend.num_qubits}")
        print(f"      - Estado: {'🟢 Disponible' if status.operational else '🔴 No disponible'}")
        if status.operational:
            print(f"      - Cola: {status.pending_jobs} trabajos")
    
    return service

service = setup_ibm_quantum(
       token='HRJvvoclqftlp0b3EyaqOdJkiNLPN3aSkoJdMWog8P2g',
       instance='demo'
   )

def normalize_kernel_matrix(K_train, K_test=None):
    """Normaliza matrices de kernel."""
    diag_train = np.sqrt(np.diag(K_train))
    diag_train = np.where(diag_train == 0, 1e-10, diag_train)
    K_train_norm = K_train / np.outer(diag_train, diag_train)
    
    if K_test is None:
        return K_train_norm
    
    assert K_test.shape[1] == K_train.shape[0], \
        f"K_test debe ser (n_test, n_train), got {K_test.shape}"
    
    if K_test.shape[0] == K_test.shape[1]:
        diag_test = np.sqrt(np.diag(K_test))
    else:
        diag_test = np.sqrt(np.diag(K_test @ K_test.T))
    
    diag_test = np.where(diag_test == 0, 1e-10, diag_test)
    K_test_norm = K_test / np.outer(diag_test, diag_train)
    
    return K_train_norm, K_test_norm


# ===============================================
# DATASET MAKE_MOONS
# ===============================================
print("="*70)
print("🌙 QUANTUM KERNEL SVM - Make Moons Dataset (IBM Quantum)")
print("="*70)

# Generar dataset
X, y = datasets.make_moons(n_samples=500, noise=0.2, random_state=42)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"\n📊 Dataset Make Moons:")
print(f"   Samples train: {len(X_train)}")
print(f"   Samples test:  {len(X_test)}")
print(f"   Features:      {X.shape[1]} (= {X.shape[1]} qubits)")
print(f"   Classes:       {len(np.unique(y))} (binary)")
print(f"   Evaluaciones:  ~{len(X_train)**2 + len(X_test)*len(X_train):,}")


# ===============================================
# OPCIÓN 1: SIMULADOR LOCAL (RÁPIDO - PARA TESTING)
# ===============================================
print("\n" + "="*70)
print("⚙️  OPCIÓN 1: Simulador Local (Recomendado para testing)")
print("="*70)

def run_with_local_simulator():
    """Ejecuta con simulador local de Qiskit."""
    from qiskit.primitives import StatevectorSampler
    from qiskit_machine_learning.state_fidelities import ComputeUncompute
    
    print("\n🔄 Configurando simulador local...")
    
    # Feature map
    feature_map = ZZFeatureMap(
        feature_dimension=2,
        reps=2,
        entanglement='full'
    )
    
    # Sampler local
    sampler = StatevectorSampler()
    fidelity = ComputeUncompute(sampler=sampler)
    
    quantum_kernel = FidelityQuantumKernel(
        feature_map=feature_map,
        fidelity=fidelity
    )
    
    print("\n🔄 Evaluando kernel cuántico (local)...")
    start = time.time()
    
    # Kernel de entrenamiento
    print(f"   [1/2] K_train ({len(X_train)}×{len(X_train)})...")
    t0 = time.time()
    K_train_raw = quantum_kernel.evaluate(X_train)
    t1 = time.time() - t0
    print(f"         ✅ {t1:.2f}s | {len(X_train)**2/t1:.0f} fid/seg")
    
    # Kernel de test
    print(f"   [2/2] K_test ({len(X_test)}×{len(X_train)})...")
    t0 = time.time()
    K_test_raw = quantum_kernel.evaluate(X_test, X_train)
    t2 = time.time() - t0
    print(f"         ✅ {t2:.2f}s | {len(X_test)*len(X_train)/t2:.0f} fid/seg")
    
    total_time = time.time() - start
    print(f"\n   ⏱️  Tiempo total: {total_time:.2f}s")
    
    # Normalización
    K_train_q, K_test_q = normalize_kernel_matrix(K_train_raw, K_test_raw)
    
    # Entrenar SVM
    print("\n🔄 Entrenando SVM con kernel cuántico...")
    svc_q = SVC(kernel="precomputed")
    svc_q.fit(K_train_q, y_train)
    preds_q = svc_q.predict(K_test_q)
    acc_q = accuracy_score(y_test, preds_q)
    
    return svc_q, acc_q, total_time, K_train_q, K_test_q


# ===============================================
# OPCIÓN 2: IBM QUANTUM HARDWARE (CON TRANSPILACIÓN)
# ===============================================
print("\n" + "="*70)
print("⚙️  OPCIÓN 2: IBM Quantum Hardware Real")
print("="*70)

def run_with_ibm_quantum(backend_name='ibm_brisbane', use_session=False):
    """
    Ejecuta en hardware real de IBM Quantum con transpilación correcta.
    
    Args:
        backend_name: Nombre del backend (ej: 'ibm_brisbane', 'ibm_kyoto')
        use_session: Si True, usa Session para optimizar ejecución
    """
    
    print("\n⚠️  CONFIGURACIÓN REQUERIDA:")
    print("   Si es tu primera vez, ejecuta:")
    print("   service = setup_ibm_quantum(")
    print("       token='TU_TOKEN_DE_IBM_QUANTUM',")
    print("       instance='ibm-q/open/main'")
    print("   )\n")
    
    # Cargar servicio
    try:
        service = QiskitRuntimeService(channel="ibm_cloud")
        print("✅ Credenciales cargadas")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n💡 Obtén tu token en: https://quantum.ibm.com/")
        return None, None, None, None, None
    
    # Seleccionar backend
    backend = service.least_busy(simulator=False, operational=True)
    print(f"\n📡 Backend seleccionado: {backend.name}")
    print(f"   Qubits: {backend.num_qubits}")
    status = backend.status()
    print(f"   Estado: {status.status_msg}")
    print(f"   Cola: {status.pending_jobs} trabajos")
    
    # Feature map
    feature_map = ZZFeatureMap(
        feature_dimension=2,
        reps=2,
        entanglement='full'
    )
    
    print("\n🔄 Configurando transpilación para hardware...")
    
    # SOLUCIÓN: Crear pass manager para transpilación
    pm = generate_preset_pass_manager(
        optimization_level=3,
        backend=backend
    )
    
    print("   ✅ Pass manager configurado")
    
    # Crear el quantum kernel con transpilación
    from qiskit_algorithms.utils import algorithm_globals
    algorithm_globals.random_seed = 42
    
    print("\n🔄 Creando Quantum Kernel con transpilación...")
    
    if use_session:
        # Usar Session para batch processing
        with Session(service=service, backend=backend) as session:
            sampler = Sampler(session=session)
            
            # CLAVE: Usar el kernel con opción de transpilación
            quantum_kernel = FidelityQuantumKernel(
                feature_map=feature_map,
            )
            
            print("\n🔄 Evaluando kernel en IBM Quantum...")
            print("   ⚠️  Esto puede tardar 5-30 minutos según la cola\n")
            
            start = time.time()
            
            # Kernel de entrenamiento
            print(f"   [1/2] K_train - Enviando a cola...")
            K_train_raw = quantum_kernel.evaluate(X_train)
            t1 = time.time() - start
            print(f"         ✅ Completado en {t1/60:.2f} min")
            
            # Kernel de test
            print(f"   [2/2] K_test - Enviando a cola...")
            t0 = time.time()
            K_test_raw = quantum_kernel.evaluate(X_test, X_train)
            t2 = time.time() - t0
            print(f"         ✅ Completado en {t2/60:.2f} min")
            
            total_time = time.time() - start
            print(f"\n   ⏱️  Tiempo total: {total_time/60:.2f} minutos")
    else:
        # Sin Session
        sampler = Sampler(mode=backend)
        
        quantum_kernel = FidelityQuantumKernel(
            feature_map=feature_map,
        )
        
        start = time.time()
        K_train_raw = quantum_kernel.evaluate(X_train)
        K_test_raw = quantum_kernel.evaluate(X_test, X_train)
        total_time = time.time() - start
    
    # Normalización
    K_train_q, K_test_q = normalize_kernel_matrix(K_train_raw, K_test_raw)
    
    # Entrenar SVM
    print("\n🔄 Entrenando SVM...")
    svc_q = SVC(kernel="precomputed")
    svc_q.fit(K_train_q, y_train)
    preds_q = svc_q.predict(K_test_q)
    acc_q = accuracy_score(y_test, preds_q)
    
    return svc_q, acc_q, total_time, K_train_q, K_test_q


# ===============================================
# EJECUTAR EXPERIMENTO
# ===============================================
print("\n" + "="*70)
print("🚀 EJECUTANDO EXPERIMENTO")
print("="*70)

# CONFIGURACIÓN: Cambia esto según necesites

print("\n✅ Modo: IBM Quantum Hardware")    

svc_quantum, acc_quantum, quantum_time, K_train_q, K_test_q = run_with_ibm_quantum(
    backend_name='ibm_torino',  # O 'ibm_kyoto', 'ibm_osaka'
    use_session=False
)
    
if svc_quantum is None:
    print("\n❌ No se pudo ejecutar. Configura tus credenciales primero.")
    exit()


# ===============================================
# KERNELS CLÁSICOS PARA COMPARACIÓN
# ===============================================
print("\n" + "="*70)
print("🖥️  KERNELS CLÁSICOS")
print("="*70)

print("\n[1/3] RBF Kernel...")
svc_rbf = SVC(kernel="rbf", gamma="scale")
svc_rbf.fit(X_train, y_train)
acc_rbf = accuracy_score(y_test, svc_rbf.predict(X_test))
print(f"      ✅ Accuracy: {acc_rbf:.3f}")

print("[2/3] Polynomial Kernel...")
svc_poly = SVC(kernel="poly", degree=3)
svc_poly.fit(X_train, y_train)
acc_poly = accuracy_score(y_test, svc_poly.predict(X_test))
print(f"      ✅ Accuracy: {acc_poly:.3f}")

print("[3/3] Linear Kernel...")
svc_lin = SVC(kernel="linear")
svc_lin.fit(X_train, y_train)
acc_lin = accuracy_score(y_test, svc_lin.predict(X_test))
print(f"      ✅ Accuracy: {acc_lin:.3f}")


# ===============================================
# RESULTADOS FINALES
# ===============================================
print("\n" + "="*70)
print("📊 RESULTADOS FINALES")
print("="*70)

print(f"\n{'Modelo':<30} {'Accuracy':>10} {'Tiempo':>15}")
print("-"*70)
tiempo_str = f"{quantum_time:.2f}s" if quantum_time < 120 else f"{quantum_time/60:.2f}min"
print(f"{'Quantum Kernel (IBM/Local)':<30} {acc_quantum:>10.3f} {tiempo_str:>15}")
print(f"{'RBF Kernel':<30} {acc_rbf:>10.3f} {'<1s':>15}")
print(f"{'Polynomial Kernel (deg=3)':<30} {acc_poly:>10.3f} {'<1s':>15}")
print(f"{'Linear Kernel':<30} {acc_lin:>10.3f} {'<1s':>15}")

print("\n" + "="*70)
print("💡 ANÁLISIS")
print("="*70)

best_classical = max(acc_rbf, acc_poly, acc_lin)
best_classical_name = ["RBF", "Poly", "Linear"][[acc_rbf, acc_poly, acc_lin].index(best_classical)]

print(f"\n🎯 Dataset: Make Moons (500 samples, noise=0.2)")
print(f"⚛️  Quantum Kernel: {acc_quantum:.3f}")
print(f"🖥️  Mejor Clásico ({best_classical_name}): {best_classical:.3f}")

if acc_quantum > best_classical + 0.01:
    diff = acc_quantum - best_classical
    print(f"\n✨ ¡Quantum kernel SUPERA a clásicos!")
    print(f"   Ventaja: +{diff:.3f} ({diff/best_classical*100:.1f}%)")
elif abs(acc_quantum - best_classical) <= 0.01:
    print(f"\n⚖️  Rendimiento equivalente al mejor clásico")
else:
    diff = best_classical - acc_quantum
    print(f"\n📊 Mejor clásico ligeramente superior")
    print(f"   Diferencia: {diff:.3f}")

# Reporte detallado
print("\n" + "="*70)
print("📋 REPORTE DETALLADO - QUANTUM KERNEL")
print("="*70)
print("\nClassification Report:")
print(classification_report(y_test, svc_quantum.predict(K_test_q), 
                          target_names=['Class 0', 'Class 1']))

print("\n" + "="*70)
print("✅ EXPERIMENTO COMPLETADO")
print("="*70)


# ===============================================
# GUÍA DE USO
# ===============================================
print("\n" + "="*70)
print("📚 GUÍA DE USO")
print("="*70)
print("""
🔹 SIMULADOR LOCAL (Rápido - Para desarrollo):
   - USE_LOCAL_SIMULATOR = True
   - Tiempo: ~10-30 segundos
   - No requiere cuenta IBM

🔹 IBM QUANTUM HARDWARE (Producción):
   1. Obtén token: https://quantum.ibm.com/
   2. Primera vez, ejecuta:
      service = setup_ibm_quantum(
          token='tu_token',
          instance='ibm-q/open/main'
      )
   3. USE_LOCAL_SIMULATOR = False
   4. Espera: 5-30 minutos

🔹 SOLUCIÓN AL ERROR DE TRANSPILACIÓN:
   ✅ Este código ya incluye la transpilación correcta
   ✅ El circuito se adapta automáticamente al hardware
   ✅ Compatible con todas las versiones de Qiskit >= 1.0

🔹 BACKENDS RECOMENDADOS:
   - ibm_brisbane (127 qubits)
   - ibm_kyoto (127 qubits)
   - ibm_osaka (127 qubits)
   - ibm_sherbrooke (127 qubits)
""")

ImportError: cannot import name 'Sampler' from 'qiskit.primitives' (c:\Users\juanc\Desktop\Cuantica\Quantum_Kernel_Comparison\.venv\Lib\site-packages\qiskit\primitives\__init__.py)